In [1]:
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
!pip install -q https://github.com/pfnet-research/xfeat/archive/master.zip

In [3]:
import pandas as pd
from sklearn.model_selection import KFold
import xfeat
import cudf



xfeat.utils.cudf_is_available()

True

In [4]:
train.columns

NameError: name 'train' is not defined

In [5]:
PATH_TRAIN_CSV = "../input/gci-homecredit/train.csv"
PATH_TEST_CSV = "../input/gci-homecredit/test.csv"

USECOLS = ['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
       'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR']

def preload():
    # Convert dataset into feather format.
    xfeat.utils.compress_df(pd.concat([
        pd.read_csv(PATH_TRAIN_CSV),
        pd.read_csv(PATH_TEST_CSV),
    ], sort=False)).reset_index(drop=True)[USECOLS].to_feather(
        "../working/train_test.ftr")


preload()

In [6]:
pd.read_feather("../working/train_test.ftr").head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0.0,Cash loans,F,N,N,0,112500.0,755190.0,36328.5,675000.0,...,NaN,0.0,0.0,0.0,0.0,-292.0,NaN,NaN,NaN,NaN
1,0.0,Cash loans,F,N,Y,0,225000.0,585000.0,16893.0,585000.0,...,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0
2,0.0,Cash loans,F,N,Y,0,54000.0,334152.0,18256.5,270000.0,...,NaN,4.0,0.0,4.0,0.0,-542.0,NaN,NaN,NaN,NaN
3,0.0,Cash loans,F,N,Y,0,67500.0,152820.0,8901.0,135000.0,...,0.767523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,Cash loans,M,N,N,0,157500.0,271066.5,21546.0,234000.0,...,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0


In [7]:
print("(1) Save numerical features")
xfeat.SelectNumerical().fit_transform(
    pd.read_feather("../working/train_test.ftr")
).reset_index(drop=True).to_feather("../working/feature_num_features.ftr")

(1) Save numerical features


In [8]:
pd.read_feather("../working/feature_num_features.ftr").head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0.0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878,-333.0,...,NaN,0.0,0.0,0.0,0.0,-292.0,NaN,NaN,NaN,NaN
1,0.0,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,365243,-4469.0,...,0.553165,0.0,0.0,0.0,0.0,-617.0,0.0,0.0,0.0,1.0
2,0.0,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523,-3640.0,...,NaN,4.0,0.0,4.0,0.0,-542.0,NaN,NaN,NaN,NaN
3,0.0,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,365243,-4950.0,...,0.767523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697,-5101.0,...,0.669057,0.0,0.0,0.0,0.0,-1243.0,0.0,0.0,0.0,4.0


In [9]:
print("(2) Categorical encoding using label encoding: 13 features")
xfeat.Pipeline([
    xfeat.SelectCategorical(), xfeat.LabelEncoder(output_suffix="")]
).fit_transform(
    pd.read_feather("../working/train_test.ftr")
).reset_index(drop=True).to_feather("../working/feature_1way_label_encoding.ftr")

(2) Categorical encoding using label encoding: 13 features


In [10]:
pd.read_feather("../working/feature_1way_label_encoding.ftr").head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,ORGANIZATION_TYPE
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,1,0,0,-1,1
2,0,0,0,1,1,2,1,0,0,0,2
3,0,0,0,1,2,1,2,1,0,-1,1
4,0,1,0,0,0,3,1,0,1,1,3


In [11]:
def target_encoding(path_source_dataframe, path_output_dataframe):
    df_train_test = pd.read_feather(path_source_dataframe)
    df_train_test.loc[:, "TARGET"] = pd.read_feather("../working/train_test.ftr")["TARGET"]
    df_train = df_train_test.dropna(subset=["TARGET"]).copy()
    df_test = df_train_test.loc[df_train_test["TARGET"].isnull()].copy()

    fold = KFold(n_splits=5, shuffle=True, random_state=111)
    encoder = xfeat.TargetEncoder(fold=fold, target_col="TARGET", output_suffix="_te")
    df_train = encoder.fit_transform(cudf.from_pandas(df_train))
    df_test = encoder.transform(cudf.from_pandas(df_test))

    pd.concat([df_train.to_pandas(), df_test.to_pandas()], sort=False).drop(
        "TARGET", axis=1).reset_index(drop=True).to_feather(path_output_dataframe)


print("(2') Target encoding of categorical variables")
target_encoding("../working/feature_1way_label_encoding.ftr",
                "../working/feature_1way_label_encoding_with_te.ftr")

(2') Target encoding of categorical variables


In [12]:
pd.read_feather("../working/feature_1way_label_encoding_with_te.ftr").head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,CODE_GENDER_te,FLAG_OWN_CAR_te,FLAG_OWN_REALTY_te,NAME_TYPE_SUITE_te,NAME_INCOME_TYPE_te,NAME_EDUCATION_TYPE_te,NAME_FAMILY_STATUS_te,NAME_HOUSING_TYPE_te,OCCUPATION_TYPE_te,ORGANIZATION_TYPE_te
0,0,0,0,0,0,0,0,0,0,0,...,0.069788,0.085127,0.085008,0.081880,0.095416,0.052917,0.075387,0.077808,0.062531,0.061260
1,0,0,0,1,0,1,1,0,0,-1,...,0.069534,0.084795,0.078885,0.081760,0.054429,0.088713,0.075429,0.077373,0.065544,0.054508
2,0,0,0,1,1,2,1,0,0,0,...,0.070077,0.085577,0.079540,0.074070,0.057623,0.089031,0.075444,0.077871,0.063632,0.080169
3,0,0,0,1,2,1,2,1,0,-1,...,0.070131,0.085773,0.079780,0.067043,0.053836,0.115890,0.056662,0.077935,0.065568,0.053911
4,0,1,0,0,0,3,1,0,1,1,...,0.101275,0.085773,0.086346,0.081924,0.074594,0.089419,0.075756,0.118218,0.118753,0.094047


In [13]:
print("(3) 2-order combination of categorical features: 78 features (13 * 12 / 2 = 78)")
xfeat.Pipeline([
    xfeat.SelectCategorical(),
    xfeat.ConcatCombination(drop_origin=True, r=2),
    xfeat.LabelEncoder(output_suffix=""),
]).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_2way_label_encoding.ftr"
)

(3) 2-order combination of categorical features: 78 features (13 * 12 / 2 = 78)


In [14]:
pd.read_feather("../working/feature_2way_label_encoding.ftr").head()

,NAME_CONTRACT_TYPECODE_GENDER_combi,NAME_CONTRACT_TYPEFLAG_OWN_CAR_combi,NAME_CONTRACT_TYPEFLAG_OWN_REALTY_combi,NAME_CONTRACT_TYPENAME_TYPE_SUITE_combi,NAME_CONTRACT_TYPENAME_INCOME_TYPE_combi,NAME_CONTRACT_TYPENAME_EDUCATION_TYPE_combi,NAME_CONTRACT_TYPENAME_FAMILY_STATUS_combi,NAME_CONTRACT_TYPENAME_HOUSING_TYPE_combi,NAME_CONTRACT_TYPEOCCUPATION_TYPE_combi,NAME_CONTRACT_TYPEORGANIZATION_TYPE_combi,...,NAME_EDUCATION_TYPENAME_FAMILY_STATUS_combi,NAME_EDUCATION_TYPENAME_HOUSING_TYPE_combi,NAME_EDUCATION_TYPEOCCUPATION_TYPE_combi,NAME_EDUCATION_TYPEORGANIZATION_TYPE_combi,NAME_FAMILY_STATUSNAME_HOUSING_TYPE_combi,NAME_FAMILY_STATUSOCCUPATION_TYPE_combi,NAME_FAMILY_STATUSORGANIZATION_TYPE_combi,NAME_HOUSING_TYPEOCCUPATION_TYPE_combi,NAME_HOUSING_TYPEORGANIZATION_TYPE_combi,OCCUPATION_TYPEORGANIZATION_TYPE_combi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,1,0,0,1,1,...,1,1,1,1,0,1,1,1,1,1
2,0,0,1,1,2,1,0,0,0,2,...,1,1,2,2,0,0,2,0,2,2
3,0,0,1,2,1,2,1,0,1,1,...,2,2,3,3,1,2,3,1,1,1
4,1,0,0,0,3,1,0,1,2,3,...,1,3,4,4,2,3,4,2,3,3


In [15]:
print("(4) 3-order combination of categorical features")
# Use `include_cols=` kwargs to reduce the total count of combinations.
# 66 features (12 * 11 / 2 = 66)
xfeat.Pipeline([
    xfeat.SelectCategorical(),
    xfeat.ConcatCombination(drop_origin=True, include_cols=["CODE_GENDER"], r=3),
    xfeat.LabelEncoder(output_suffix=""),
]).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_3way_including_CODE_GENDER_label_encoding.ftr"
)

(4) 3-order combination of categorical features


In [16]:
pd.read_feather("../working/feature_3way_including_CODE_GENDER_label_encoding.ftr").head()

,CODE_GENDERNAME_CONTRACT_TYPEFLAG_OWN_CAR_combi,CODE_GENDERNAME_CONTRACT_TYPEFLAG_OWN_REALTY_combi,CODE_GENDERNAME_CONTRACT_TYPENAME_TYPE_SUITE_combi,CODE_GENDERNAME_CONTRACT_TYPENAME_INCOME_TYPE_combi,CODE_GENDERNAME_CONTRACT_TYPENAME_EDUCATION_TYPE_combi,CODE_GENDERNAME_CONTRACT_TYPENAME_FAMILY_STATUS_combi,CODE_GENDERNAME_CONTRACT_TYPENAME_HOUSING_TYPE_combi,CODE_GENDERNAME_CONTRACT_TYPEOCCUPATION_TYPE_combi,CODE_GENDERNAME_CONTRACT_TYPEORGANIZATION_TYPE_combi,CODE_GENDERFLAG_OWN_CARFLAG_OWN_REALTY_combi,...,CODE_GENDERNAME_EDUCATION_TYPENAME_FAMILY_STATUS_combi,CODE_GENDERNAME_EDUCATION_TYPENAME_HOUSING_TYPE_combi,CODE_GENDERNAME_EDUCATION_TYPEOCCUPATION_TYPE_combi,CODE_GENDERNAME_EDUCATION_TYPEORGANIZATION_TYPE_combi,CODE_GENDERNAME_FAMILY_STATUSNAME_HOUSING_TYPE_combi,CODE_GENDERNAME_FAMILY_STATUSOCCUPATION_TYPE_combi,CODE_GENDERNAME_FAMILY_STATUSORGANIZATION_TYPE_combi,CODE_GENDERNAME_HOUSING_TYPEOCCUPATION_TYPE_combi,CODE_GENDERNAME_HOUSING_TYPEORGANIZATION_TYPE_combi,CODE_GENDEROCCUPATION_TYPEORGANIZATION_TYPE_combi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,0,1,1,1,...,1,1,1,1,0,1,1,1,1,1
2,0,1,1,2,1,0,0,0,2,1,...,1,1,2,2,0,0,2,0,2,2
3,0,1,2,1,2,1,0,1,1,1,...,2,2,3,3,1,2,3,1,1,1
4,1,2,3,3,3,2,1,2,3,2,...,3,3,4,4,2,3,4,2,3,3


In [17]:
print("(5) Convert numerical to categorical using round: 12 features")
df_rnum = (
    xfeat.Pipeline(
        [
            xfeat.SelectNumerical(),
            xfeat.LambdaEncoder(
                lambda x: str(x)[:-2],
                output_suffix="_rnum",
                exclude_cols=["TARGET"],
            ),
        ]
    )
    .fit_transform(pd.read_feather("../working/train_test.ftr"))
    .reset_index(drop=True)
)
df_rnum.to_feather("../working/feature_round_num.ftr")
rnum_cols = [col for col in df_rnum.columns if col.endswith("_rnum")]
xfeat.Pipeline([xfeat.LabelEncoder(output_suffix="")]).fit_transform(
    pd.read_feather("../working/feature_round_num.ftr")[rnum_cols]
).reset_index(drop=True).to_feather("../working/feature_round_num_label_encoding.ftr")




(5) Convert numerical to categorical using round: 12 features


In [18]:
pd.read_feather("../working/feature_round_num_label_encoding.ftr").head()

,CNT_CHILDREN_rnum,AMT_INCOME_TOTAL_rnum,AMT_CREDIT_rnum,AMT_ANNUITY_rnum,AMT_GOODS_PRICE_rnum,REGION_POPULATION_RELATIVE_rnum,DAYS_BIRTH_rnum,DAYS_EMPLOYED_rnum,DAYS_REGISTRATION_rnum,DAYS_ID_PUBLISH_rnum,...,EXT_SOURCE_3_rnum,OBS_30_CNT_SOCIAL_CIRCLE_rnum,DEF_30_CNT_SOCIAL_CIRCLE_rnum,OBS_60_CNT_SOCIAL_CIRCLE_rnum,DEF_60_CNT_SOCIAL_CIRCLE_rnum,DAYS_LAST_PHONE_CHANGE_rnum,AMT_REQ_CREDIT_BUREAU_HOUR_rnum,AMT_REQ_CREDIT_BUREAU_MON_rnum,AMT_REQ_CREDIT_BUREAU_QRT_rnum,AMT_REQ_CREDIT_BUREAU_YEAR_rnum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,1,1,1,1,1
2,0,2,2,2,2,2,2,2,2,2,...,0,1,0,1,0,2,0,0,0,0
3,0,3,3,3,3,3,3,1,3,3,...,2,0,0,0,0,3,1,1,1,2
4,0,4,4,4,4,4,4,3,4,4,...,3,0,0,0,0,4,1,1,1,3


In [19]:
print("(6) 2-order Arithmetic combinations.")
xfeat.Pipeline(
    [
        xfeat.SelectNumerical(),
        xfeat.ArithmeticCombinations(
            exclude_cols=["TARGET"], drop_origin=True, operator="+", r=2,output_suffix="+"
        ),
    ]
).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_arithmetic+_combi2.ftr"
)
xfeat.Pipeline(
    [
        xfeat.SelectNumerical(),
        xfeat.ArithmeticCombinations(
            exclude_cols=["TARGET"], drop_origin=True, operator="-", r=2,output_suffix="-"
        ),
    ]
).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_arithmetic-_combi2.ftr"
)

xfeat.Pipeline(
    [
        xfeat.SelectNumerical(),
        xfeat.ArithmeticCombinations(
            exclude_cols=["TARGET"], drop_origin=True, operator="*", r=2,output_suffix="*"
        ),
    ]
).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_arithmetic*_combi2.ftr"
)

xfeat.Pipeline(
    [
        xfeat.SelectNumerical(),
        xfeat.ArithmeticCombinations(
            exclude_cols=["TARGET"], drop_origin=True, operator="/", r=2,output_suffix="/"
        ),
    ]
).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_arithmeticdiv_combi2.ftr"
)

(6) 2-order Arithmetic combinations.


In [20]:
pd.read_feather("../working/feature_arithmetic+_combi2.ftr").head()

,CNT_CHILDRENAMT_INCOME_TOTAL+,CNT_CHILDRENAMT_CREDIT+,CNT_CHILDRENAMT_ANNUITY+,CNT_CHILDRENAMT_GOODS_PRICE+,CNT_CHILDRENREGION_POPULATION_RELATIVE+,CNT_CHILDRENDAYS_BIRTH+,CNT_CHILDRENDAYS_EMPLOYED+,CNT_CHILDRENDAYS_REGISTRATION+,CNT_CHILDRENDAYS_ID_PUBLISH+,CNT_CHILDRENOWN_CAR_AGE+,...,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_HOUR+,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_MON+,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_QRT+,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_YEAR+,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_MON+,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_QRT+,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_YEAR+,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_QRT+,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_YEAR+,AMT_REQ_CREDIT_BUREAU_QRTAMT_REQ_CREDIT_BUREAU_YEAR+
0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878,-333.0,-522,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,365243,-4469.0,-3436,NaN,...,-617.0,-617.0,-617.0,-616.0,0.0,0.0,1.0,0.0,1.0,1.0
2,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523,-3640.0,-2050,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,365243,-4950.0,-3951,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697,-5101.0,-3226,NaN,...,-1243.0,-1243.0,-1243.0,-1239.0,0.0,0.0,4.0,0.0,4.0,4.0


In [21]:
pd.read_feather("../working/feature_arithmetic-_combi2.ftr").head()

,CNT_CHILDRENAMT_INCOME_TOTAL-,CNT_CHILDRENAMT_CREDIT-,CNT_CHILDRENAMT_ANNUITY-,CNT_CHILDRENAMT_GOODS_PRICE-,CNT_CHILDRENREGION_POPULATION_RELATIVE-,CNT_CHILDRENDAYS_BIRTH-,CNT_CHILDRENDAYS_EMPLOYED-,CNT_CHILDRENDAYS_REGISTRATION-,CNT_CHILDRENDAYS_ID_PUBLISH-,CNT_CHILDRENOWN_CAR_AGE-,...,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_HOUR-,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_MON-,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_QRT-,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_YEAR-,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_MON-,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_QRT-,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_YEAR-,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_QRT-,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_YEAR-,AMT_REQ_CREDIT_BUREAU_QRTAMT_REQ_CREDIT_BUREAU_YEAR-
0,-112500.0,-755190.0,-36328.5,-675000.0,-0.010032,9233,878,333.0,522,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-225000.0,-585000.0,-16893.0,-585000.0,-0.008019,20148,-365243,4469.0,3436,NaN,...,-617.0,-617.0,-617.0,-618.0,0.0,0.0,-1.0,0.0,-1.0,-1.0
2,-54000.0,-334152.0,-18256.5,-270000.0,-0.004960,18496,523,3640.0,2050,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-67500.0,-152820.0,-8901.0,-135000.0,-0.005002,24177,-365243,4950.0,3951,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-157500.0,-271066.5,-21546.0,-234000.0,-0.006296,10685,697,5101.0,3226,NaN,...,-1243.0,-1243.0,-1243.0,-1247.0,0.0,0.0,-4.0,0.0,-4.0,-4.0


In [22]:
pd.read_feather("../working/feature_arithmetic*_combi2.ftr").head()

,CNT_CHILDRENAMT_INCOME_TOTAL*,CNT_CHILDRENAMT_CREDIT*,CNT_CHILDRENAMT_ANNUITY*,CNT_CHILDRENAMT_GOODS_PRICE*,CNT_CHILDRENREGION_POPULATION_RELATIVE*,CNT_CHILDRENDAYS_BIRTH*,CNT_CHILDRENDAYS_EMPLOYED*,CNT_CHILDRENDAYS_REGISTRATION*,CNT_CHILDRENDAYS_ID_PUBLISH*,CNT_CHILDRENOWN_CAR_AGE*,...,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_HOUR*,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_MON*,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_QRT*,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_YEAR*,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_MON*,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_QRT*,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_YEAR*,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_QRT*,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_YEAR*,AMT_REQ_CREDIT_BUREAU_QRTAMT_REQ_CREDIT_BUREAU_YEAR*
0,0.0,0.0,0.0,0.0,0.0,0,0,-0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0,0,-0.0,0,NaN,...,-0.0,-0.0,-0.0,-617.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0,-0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0,0,-0.0,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0,-0.0,0,NaN,...,-0.0,-0.0,-0.0,-4972.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
pd.read_feather("../working/feature_arithmeticdiv_combi2.ftr").head()

,CNT_CHILDRENAMT_INCOME_TOTAL/,CNT_CHILDRENAMT_CREDIT/,CNT_CHILDRENAMT_ANNUITY/,CNT_CHILDRENAMT_GOODS_PRICE/,CNT_CHILDRENREGION_POPULATION_RELATIVE/,CNT_CHILDRENDAYS_BIRTH/,CNT_CHILDRENDAYS_EMPLOYED/,CNT_CHILDRENDAYS_REGISTRATION/,CNT_CHILDRENDAYS_ID_PUBLISH/,CNT_CHILDRENOWN_CAR_AGE/,...,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_HOUR/,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_MON/,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_QRT/,DAYS_LAST_PHONE_CHANGEAMT_REQ_CREDIT_BUREAU_YEAR/,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_MON/,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_QRT/,AMT_REQ_CREDIT_BUREAU_HOURAMT_REQ_CREDIT_BUREAU_YEAR/,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_QRT/,AMT_REQ_CREDIT_BUREAU_MONAMT_REQ_CREDIT_BUREAU_YEAR/,AMT_REQ_CREDIT_BUREAU_QRTAMT_REQ_CREDIT_BUREAU_YEAR/
0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,NaN,...,-inf,-inf,-inf,-617.00,NaN,NaN,0.0,NaN,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,NaN,...,-inf,-inf,-inf,-310.75,NaN,NaN,0.0,NaN,0.0,0.0


In [24]:
"""print("(7) Add more combinations: 4-order concat combinations.")
xfeat.Pipeline(
    [
        xfeat.SelectCategorical(),
        xfeat.ConcatCombination(drop_origin=True, include_cols=["CODE_GENDER"], r=4),
        xfeat.LabelEncoder(output_suffix=""),
    ]
).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(
    drop=True
).to_feather(
    "../working/feature_4way_including_CODE_GENDER_label_encoding.ftr"
)"""



'print("(7) Add more combinations: 4-order concat combinations.")\nxfeat.Pipeline(\n    [\n        xfeat.SelectCategorical(),\n        xfeat.ConcatCombination(drop_origin=True, include_cols=["CODE_GENDER"], r=4),\n        xfeat.LabelEncoder(output_suffix=""),\n    ]\n).fit_transform(pd.read_feather("../working/train_test.ftr")).reset_index(\n    drop=True\n).to_feather(\n    "../working/feature_4way_including_CODE_GENDER_label_encoding.ftr"\n)'

In [25]:
print("(3') Target encoding of categorical variables")
target_encoding("../working/feature_2way_label_encoding.ftr",
                "../working/feature_2way_label_encoding_with_te.ftr")

print("(4') Target encoding of categorical variables")
target_encoding("../working/feature_3way_including_CODE_GENDER_label_encoding.ftr",
                "../working/feature_3way_including_CODE_GENDER_label_encoding_with_te.ftr")
"""
print("(5') Target encoding of categorical variables")
target_encoding("../working/feature_round_num_label_encoding.ftr",
                "../working/feature_round_num_label_encoding_with_te.ftr")

print("(7') Target encoding of categorical variables")
target_encoding("../working/feature_11way_including_v22_label_encoding.ftr",
                "../working/feature_11way_including_v22_label_encoding_with_te.ftr")"""

(3') Target encoding of categorical variables
(4') Target encoding of categorical variables


'\nprint("(5\') Target encoding of categorical variables")\ntarget_encoding("../working/feature_round_num_label_encoding.ftr",\n                "../working/feature_round_num_label_encoding_with_te.ftr")\n\nprint("(7\') Target encoding of categorical variables")\ntarget_encoding("../working/feature_11way_including_v22_label_encoding.ftr",\n                "../working/feature_11way_including_v22_label_encoding_with_te.ftr")'

In [26]:
!ls -lha ../working/

total 3.2G
drwxr-xr-x 2 root root 4.0K Jun 25 20:12  .
drwxr-xr-x 6 root root 4.0K Jun 25 20:08  ..
---------- 1 root root  14K Jun 25 20:08  __notebook__.ipynb
-rw-r--r-- 1 root root  20M Jun 25 20:10  feature_1way_label_encoding.ftr
-rw-r--r-- 1 root root  40M Jun 25 20:10  feature_1way_label_encoding_with_te.ftr
-rw-r--r-- 1 root root  98M Jun 25 20:10  feature_2way_label_encoding.ftr
-rw-r--r-- 1 root root 196M Jun 25 20:12  feature_2way_label_encoding_with_te.ftr
-rw-r--r-- 1 root root  80M Jun 25 20:11  feature_3way_including_CODE_GENDER_label_encoding.ftr
-rw-r--r-- 1 root root 160M Jun 25 20:12  feature_3way_including_CODE_GENDER_label_encoding_with_te.ftr
-rw-r--r-- 1 root root 554M Jun 25 20:11 'feature_arithmetic*_combi2.ftr'
-rw-r--r-- 1 root root 554M Jun 25 20:11  feature_arithmetic+_combi2.ftr
-rw-r--r-- 1 root root 554M Jun 25 20:11  feature_arithmetic-_combi2.ftr
-rw-r--r-- 1 root root 776M Jun 25 20:11  feature_arithmeticdiv_combi2.ftr
-rw-r--r-- 1 root root  25M Jun 